In [85]:
# Atnaujintas variantas, po to, kaip padaryti pakeitimai failų varduose 2022 03 26
#
# Skriptas zive EKG pūpsnių CNN VU klasifikatoriaus testavimui ir tikslumo įvertinimui, funkcijos paimamos iš
#  aplanko zive_cnn_fda_vu_v1.py, modelis iš model_cnn_fda_vu_v1, testuojami duomenys iš VU_DUOM 50 pacientų
# įrašų saugyklos, jame yra ir failas all_beats_attr
 
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import sys, os
from pathlib import Path
from icecream import ic
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_recall_fscore_support

from zive_util_vu import cm2df, show_confusion_matrix 
from zive_util_vu import create_dir, create_subdir, get_rev_dictionary
from zive_util_vu import runtime, split_SubjCode
from zive_util_vu import get_freq_unique_values, zive_read_file_1ch

from zive_util_vu import get_beat_attributes
from zive_util_vu import get_userId, get_rec_Id 
from zive_util_vu import confusion_matrix_modified, zive_read_df_rpeaks

from zive_cnn_fda_vu_v1 import predict_cnn_fda_vu_v1, classify_cnn_fda_vu_v1, get_pred_symbols

print(tf.__version__)

print("Skriptas zive-arrh EKG segmentų apmokyto klasifikatoriaus tikslumo įvertinimui")
print('Modelis CNN VU su EKG sekos reikšmėmis, EKG formos požymiais, RR intervalais prieš ir po R dantelio')

import warnings
# warnings.filterwarnings("ignore")

my_os=sys.platform
print("OS in my system : ",my_os)

if my_os != 'linux':
    OS = 'Windows'
else:  
    OS = 'Ubuntu'

# Pasiruošimas

# //////////////// NURODOMI PARAMETRAI /////////////////////////////////////////////////////

# Bendras duomenų aplankas, kuriame patalpintas subfolderis name_db

if OS == 'Windows':
    Duomenu_aplankas = 'D:\DI\Data\MIT&ZIVE'   # variantas: Windows
else:
    Duomenu_aplankas = '/home/kesju/DI/Data/MIT&ZIVE'   # arba variantas: UBUNTU, be Docker

# jei variantas Docker pasirenkame:
# Duomenu_aplankas = '/Data/MIT&ZIVE'

# Vietinės talpyklos aplankas ir pūpsnių atributų failas
db_folder = 'ZIVE_BUFFER_2022_03_24'

# Vietinės talpyklos aplankas ir pūpsnių atributų failas
all_beats_attr_fname = 'all_beats_attr_z.csv'

# Failai pūpsnių klasių formavimui
selected_beats = {'N':0, 'S':1, 'V':2}
all_beats =  {'N':0, 'S':1, 'V':2, 'U':3, 'F':3}  

# Diskretizavimo dažnis:
fs = 200

# /////////////////////////////////////////////////////////////////

#  Nuoroda į aplanką su MIT2ZIVE duomenų rinkiniu
db_path = Path(Duomenu_aplankas, db_folder)

# Nuoroda į aplanką su EKG įrašais (.npy) ir anotacijomis (.json)
rec_dir = Path(db_path, 'records_selected')

# Nuoroda į modelio aplanką
# model_dir = Path(Duomenu_aplankas, 'DNN', 'best_models', 'all_ft')
model_dir = 'model_cnn_fda_vu_v1'

# Išvedame parametrus
print("\nBendras duomenų aplankas: ", Duomenu_aplankas)
print("Zive duomenų aplankas: ", db_folder)
print("Aplankas su originaliais EKG įrašais ir anotacijomis (.json) ", rec_dir)
print("Pūpsnių atributų failas:", all_beats_attr_fname)
print("Diskretizavimo dažnis: ", fs)
print('Klasifikavimo schema:', selected_beats)
print('Klasių skaičius:', len(selected_beats))
print("Modelio ir scaler parametrai nuskaitomas iš aplanko: ", model_dir)

print("\n")


2.6.0
Skriptas zive-arrh EKG segmentų apmokyto klasifikatoriaus tikslumo įvertinimui
Modelis CNN VU su EKG sekos reikšmėmis, EKG formos požymiais, RR intervalais prieš ir po R dantelio
OS in my system :  linux

Bendras duomenų aplankas:  /home/kesju/DI/Data/MIT&ZIVE
Zive duomenų aplankas:  ZIVE_BUFFER_2022_03_24
Aplankas su originaliais EKG įrašais ir anotacijomis (.json)  /home/kesju/DI/Data/MIT&ZIVE/ZIVE_BUFFER_2022_03_24/records_selected
Pūpsnių atributų failas: all_beats_attr_z.csv
Diskretizavimo dažnis:  200
Klasifikavimo schema: {'N': 0, 'S': 1, 'V': 2}
Klasių skaičius: 3
Modelio ir scaler parametrai nuskaitomas iš aplanko:  model_cnn_fda_vu_v1




In [86]:
# PASIRUOŠIMAS

pd.set_option("display.max_rows", 6000, "display.max_columns",200)
pd.set_option('display.width', 1000)

import warnings
warnings.filterwarnings("ignore")

# Naudojamų požymių sąrašas

all_features = ['RRl', 'RRr', 'RRl/RRr',
                'signal_mean', 'signal_std', 'P_val', 'Q_val', 'R_val', 'S_val', 'T_val',
                'P_pos', 'Q_pos', 'R_pos', 'S_pos', 'T_pos', 'QRS', 'PR', 'ST', 'QT', '0', '1', '2',
                '3', '4', '5', '6', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18',
                '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32',
                '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46',
                '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60',
                '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74',
                '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88',
                '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102',
                '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114',
                '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126',
                '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138',
                '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150',
                '151', '152', '153', '154', '155', '156', '157', '158', '159', '160', '161', '162',
                '163', '164', '165', '166', '167', '168', '169', '170', '171', '172', '173', '174',
                '175', '176', '177', '178', '179', '180', '181', '182', '183', '184', '185', '186',
                '187', '188', '189', '190', '191', '192', '193', '194', '195', '196', '197', '198',
                '199']

# Pacientų įrašų kodų sąrašas iš Zive saugyklos:
SubjCodes = \
 [10000, 10001, 10002, 10003, 10010, 10020, 10021, 10022, 10023, 10024, 10025, 10030, 10031, 10032, 10033, 10034, 10035,
 10036, 10040, 10050, 10051, 10052, 10053, 10054, 10055, 10056, 10057, 10058, 10059, 10060, 10061, 10062, 10063, 10064,
 10065, 10066, 10067, 10068, 10069, 10070, 10071, 10072, 10080, 10081, 10082, 10083, 10084, 10085, 10086, 10087, 10088,
 10089, 10090, 10100, 10101, 10102, 10103, 10104, 10105, 10106, 10107, 10108, 10109, 10110, 10120, 10121, 10122, 10123,
 10124, 10125, 10126, 10127, 10128, 10129, 10130, 10131, 10132, 10133, 10134, 10140, 10141, 10142, 10143, 10144, 10145,
 10146, 10147, 10148, 10149, 10150, 10151, 10152, 10153, 10154, 10155, 10156, 10157, 10158, 10159, 10160, 10161, 10162,
 10163, 10164, 10165, 10166, 10167, 10168, 10169, 10170, 10171, 10172, 10173, 10174, 10175, 10176, 10180, 10190, 10191,
 10192, 10193, 10194, 10195, 10196, 10197, 10198, 10199, 10200, 10201, 10210, 10211, 10212, 10213, 10214, 10215, 10216,
 10217, 10220, 10221, 10230, 10231, 10232, 10233, 10234, 10235, 10236, 10237, 10238, 10239, 10240, 10241, 10250, 10260,
 10270, 10280, 10281, 10290, 10291, 10300, 10301, 10302, 10303, 10304, 10305, 10306, 10307, 10308, 10309, 10310, 10311,
 10312, 10313, 10314, 10315, 10316, 10317, 10318, 10319, 10320, 10321, 10322, 10323, 10324, 10325, 10326, 10327, 10328,
 10329, 10330, 10331, 10332, 10333, 10334, 10335, 10340, 10341, 10342, 10343, 10344, 10345, 10346, 10347, 10348, 10349,
 10350, 10351, 10352, 10353, 10354, 10355, 10356, 10360, 10361, 10362, 10370, 10371, 10380, 10381, 10382, 10390, 10400,
 10401, 10410, 10411, 10420, 10421, 10422, 10423, 10424, 10425, 10426, 10427, 10428, 10429, 10430, 10431, 10432, 10433,
 10434, 10435, 10436, 10437, 10438, 10439, 10440, 10441, 10442, 10443, 10444, 10445, 10450, 10451, 10452, 10453, 10454,
 10455, 10456, 10457, 10458, 10459, 10460, 10461, 10462, 10470, 10471, 10472, 10473, 10474, 10475, 10480, 10481, 10482,
 10483, 10484, 10485, 10486, 10487, 10488, 10489, 10490, 10491, 10492, 10493, 10494, 10495, 10496, 10497, 10498, 10499]

SubjCodes = ["1000.1631134.867", "1001.1631076.286", "1002.1630844.567", "1003.1636452.484"] #Testavimui
# SubjCodes = ["1000.1631134.867"] #Testavimui

# Išvedamas pacientų įrašų sąrašas
print("\nAtliekama pūpsnių pacientų įrašuose klasifikacija")
print("Klasifikuojamų įrašų sąrašas:", SubjCodes)

# Kas kiek išvedamas apdorotų sekų skaičius
show_period = 100

# Klasių simbolinių vardų sąrašas ir klasių skaičius
class_names = list(selected_beats.keys()) 
n_classes = len(selected_beats)
print(class_names)

# Nuskaitome pūpsnių atributų masyvą
file_path = Path(rec_dir, all_beats_attr_fname)
all_beats_attr = pd.read_csv(file_path, index_col=0, dtype = {'userNr': int, 'file_name': str,
                                                             'sample': int, 'symbol': str, 'label': int})
all_beat_indices = all_beats_attr.index

index_start = 0
validation_set_stats = pd.DataFrame(columns=['idx', 'test_label', 'pred_label', 'SubjCode'])

start_time = time.time()
# Ciklas per pacientų įrašus
for SubjCode in SubjCodes:
    userNr, file_name = split_SubjCode(SubjCode)
    # userId, recordingId =  get_rec_Id(rec_dir, userNr, recordingNr)
    print(f"\n{SubjCode} {str(file_name):>26}")

    # Paciento EKG sekų formavimas ir klasifikacija  
    file_path = Path(rec_dir, file_name) # Dirbame su originaliais duomenimis
    sign_raw = zive_read_file_1ch(file_path)  
    signal_length = sign_raw.shape[0]
    print('signal_length= ', signal_length, sign_raw.shape)
    signal = sign_raw

    # Filtruojame signalą
    # signal = signal_filter(signal=sign_raw, sampling_rate=200, lowcut=0.2, method="butterworth", order=5)

    # Nuskaitome paciento anotacijas ir jų indeksus
    df_rpeaks = zive_read_df_rpeaks(rec_dir, file_name)
    atr_sample = df_rpeaks['sampleIndex'].to_numpy()
    atr_symbol = df_rpeaks['annotationValue'].to_numpy()

    # SUFORMUOJAME TESTINĮ ir PRISKIRTŲ KLASIŲ NUMERIŲ MASYVUS

    # Nereikalingiems 'U' ir 'F' suteikiam klasę 3, kurią vėliau apvalysime  
    test_labels = np.array([all_beats[symbol] for symbol in atr_symbol])

    (unique, counts) = np.unique(test_labels, return_counts=True)
    total = counts.sum()
    print("test_labels: ", unique, counts, total)

    # Surandame pradinį SubjCode įrašo indeksą faile all_beats_attr
    userNr, file_name = split_SubjCode(SubjCode)
    selected_ind = all_beat_indices[(all_beats_attr['userNr']==userNr) & (all_beats_attr['file_name']==file_name)]
    # print(f"SubjCode: {SubjCode}  first elem: {selected_ind[0]} last elem: {selected_ind[-1]}  tot: {len(selected_ind)}")
    index_start = selected_ind[0]
    # print('\nSubjCode:',SubjCode, 'index_start:', index_start)   

    pred_labels = predict_cnn_fda_vu_v1(signal, atr_sample, model_dir)
    
    (unique, counts) = np.unique(pred_labels, return_counts=True)
    total = counts.sum()
    print("pred_labels: ",unique, counts, total)

    classification=[]
    for i, i_sample in enumerate(atr_sample):
        classification.append({'sample':i_sample, 'annotation':pred_labels[i]})
        if (pred_labels[i] != 0):
            print(f"i_sample: {i_sample} pred_label: {pred_labels[i]}")  

    #  Praleisdami indeksą, jei masyvuose test_labels ir pred_labels yra reikšmė == 3,
    # suformuojame klasifikuotinų pūpsnių indeksų sąrašą
    for idx in range(len(atr_sample)):
        flag = (test_labels[idx] == 3) or (pred_labels[idx] == 3)
        if (flag == False):
            validation_set_stats = validation_set_stats.append({'idx':index_start+idx,
            'test_label':test_labels[idx],'pred_label':pred_labels[idx], 'SubjCode': SubjCode}, ignore_index=True)

    # SURANDAME TIKSLUMO PARAMETRUS
    test_y = np.array(validation_set_stats[validation_set_stats['SubjCode']==SubjCode]['test_label']).astype('int') 

    # print(all_beats_attr.info())

    pred_y = np.array(validation_set_stats[validation_set_stats['SubjCode']==SubjCode]['pred_label']).astype('int')
    confusion = confusion_matrix(test_y, pred_y)
    # print(confusion)
    prec,rec,fsc,sup = precision_recall_fscore_support(test_y, pred_y, labels=[0, 1, 2], zero_division=0)

    str1 =f"N:{int(sup[0]):>5} S:{(int(sup[1])):3} V:{int(sup[2]):3}" 
    str2 = f"  Nprec:{prec[0]:>5.2f} Nrec:{rec[0]:5.2f} Nfsc:{fsc[0]:5.2f}"
    str3 = f"  Sprec:{prec[1]:>5.2f} Srec:{rec[1]:5.2f} Sfsc:{fsc[1]:5.2f}"
    str4 = f"  Vprec:{prec[2]:>5.2f} Vrec:{rec[2]:5.2f} Vfsc:{fsc[2]:5.2f}"
    print(str1+str2+str3+str4)

    # print(len(validation_set_stats))
    # print(len(test_y))
    # print(len(pred_y))

    end_time = time.time()
    print('\n')
    runtime(end_time-start_time)

validate_ind_lst = list(validation_set_stats['idx'])
y_validate = np.array(validation_set_stats['test_label']).astype('int')
y_predicted = np.array(validation_set_stats['pred_label']).astype('int')



Atliekama pūpsnių pacientų įrašuose klasifikacija
Klasifikuojamų įrašų sąrašas: ['1000.1631134.867']
['N', 'S', 'V']

1000.1631134.867                1631134.867
signal_length=  127999 (127999,)
test_labels:  [0 1] [656  12] 668
pred_labels:  [0 1 2 3] [623  40   2   3] 668
i_sample: 205 pred_label: 3
i_sample: 4498 pred_label: 1
i_sample: 4653 pred_label: 1
i_sample: 7407 pred_label: 1
i_sample: 7641 pred_label: 1
i_sample: 11339 pred_label: 2
i_sample: 18333 pred_label: 3
i_sample: 18947 pred_label: 1
i_sample: 19100 pred_label: 1
i_sample: 20504 pred_label: 1
i_sample: 20648 pred_label: 1
i_sample: 21872 pred_label: 1
i_sample: 22000 pred_label: 1
i_sample: 22127 pred_label: 1
i_sample: 22251 pred_label: 1
i_sample: 22625 pred_label: 1
i_sample: 22765 pred_label: 1
i_sample: 23459 pred_label: 1
i_sample: 30407 pred_label: 1
i_sample: 30555 pred_label: 1
i_sample: 34221 pred_label: 1
i_sample: 41433 pred_label: 1
i_sample: 62102 pred_label: 1
i_sample: 66679 pred_label: 1
i_sample: 

In [87]:
# MODELIO TIKSLUMO VERTINIMO IŠ VERTINIMO IMTIES REZULTATAI

print("\nMODELIO TIKSLUMO VERTINIMO iš VERTINIMO IMTIES REZULTATAI")
print("Modelis iš aplanko: ", model_dir)

cols, dist, tot = get_freq_unique_values(y_validate, ['N', 'S', 'V'])
print(f"Klasės {cols}: {dist} Suma: {tot} ")

# APIBENDRINTI REZULTATAI

print('\nAPIBENDRINTI REZULTATAI\n')

# Skaičiuojame ir išvedame klasifikavimo lentelę
confusion = confusion_matrix(y_validate, y_predicted)
pd.set_option('display.precision',3)
show_confusion_matrix(confusion, class_names)
# print('\n')

print("\nClassification Report\n")
# target_names = [key for (key, value) in selected_beats.items()]

pd.set_option("display.max_rows", 6000, "display.max_columns",200)
pd.set_option('display.width', 1000)

print(classification_report(y_validate, y_predicted, target_names=class_names, digits=3))
report = classification_report(y_validate, y_predicted, target_names=class_names, output_dict=True)
# output_dictbool, default=False, If True, return output as dict.
df_report = pd.DataFrame(report).transpose()
# https://medium.com/@asmaiya/you-can-something-like-this-84d28e0fd31f

# Įrašome į diską
filepath = Path(model_dir, 'Apibendrinti_rezultatai.csv') 
df_report.to_csv(filepath)    
print(f'\nApibendrinti_rezultatai įrašyti į:  {filepath}')



MODELIO TIKSLUMO VERTINIMO iš VERTINIMO IMTIES REZULTATAI
Modelis iš aplanko:  model_cnn_fda_vu_v1
Klasės ['N', 'S', 'V']: [653  12] Suma: 665 

APIBENDRINTI REZULTATAI

Confusion Matrix
     N   S  V
N  623  30  0
S    0  10  2
V    0   0  0


Zero values! Cannot calculate Normalized Confusion Matrix

Classification Report

              precision    recall  f1-score   support

           N      1.000     0.954     0.976       653
           S      0.250     0.833     0.385        12
           V      0.000     0.000     0.000         0

    accuracy                          0.952       665
   macro avg      0.417     0.596     0.454       665
weighted avg      0.986     0.952     0.966       665


Apibendrinti_rezultatai įrašyti į:  model_cnn_fda_vu_v1/Apibendrinti_rezultatai.csv


In [88]:
# KLAIDŲ PASISKIRSTYMAS PER PACIENTUS IR JŲ ĮRAŠUS

from zive_util_vu import zive_read_df_data, create_SubjCode
from sklearn.metrics import accuracy_score

def collect_noise_locs(rec_dir, all_beats_attr):

    noise_arr_tot = np.empty(0, dtype=int)
    
    grouped = all_beats_attr.groupby(['userNr', 'file_name'])
    for key, group in grouped:
        # print('\n',key)
        userNr = key[0]
        file_name = key[1]

        # SubjCode naudojamas atveju, kai duomenis yra formoje SubjCode.npy, SubjCode.json
        # SubjCode = create_SubjCode(userNr, file_name)
        # filepath = Path(rec_dir, SubjCode + '.json')  # Si eilute pritaikyta atvejui,
        filepath = Path(rec_dir, file_name + '.json')
        df_noises = zive_read_df_data(filepath, 'noises')

        noise_arr = np.full(shape=len(group), fill_value=0,  dtype=int)
        idx_noise = 0

        for i, row_i in group.iterrows():
            sample = row_i['sample']
            for j, row_j in df_noises.iterrows():
                if ((sample > row_j['startIndex']) & (sample < row_j['endIndex'])):
                    noise_arr[idx_noise] = 1
                    idx_noise += 1
        noise_arr_tot = np.append(noise_arr_tot, noise_arr)
    return noise_arr_tot

def get_error(y_test, y_pred):
    # Error in %
    n_errors = 0
    for idx in range(len(y_pred)):
        if (y_test[idx] != y_pred[idx]):
            n_errors += 1
    Err = float(n_errors)/len(y_pred)*100.
    Err = round(Err, 1)
    return Err

# Pasiruošimas

pd.set_option("display.max_rows", 6000, "display.max_columns", 15)
pd.set_option('display.width', 1000)

# Sukuriame ir užpildome dataframe su sekų parametrais
df_seq_errors = pd.DataFrame(columns= ['idx', 'userNr', 'file_name'])

rows_list = []
for idx in validate_ind_lst:
# Surandame  userNr, file_name, symbol
    userNr, file_name, label, symbol = get_beat_attributes(idx, all_beats_attr)
    seq_attr = {'idx': idx, 'userNr':userNr, 'file_name':file_name}
    rows_list.append(seq_attr)

# print(rows_list[:10])

# Čia įdedame informaciją, ar pūpsnys patenka į triukšmų zoną
noise_arr = collect_noise_locs(rec_dir, all_beats_attr)
noise_arr = noise_arr[validate_ind_lst]

# print(noise_arr)
# unique, counts = np.unique(noise_arr, return_counts=True)
# print(unique, counts, noise_arr.shape[0])
# print(counts.sum())

df_seq_errors = pd.DataFrame(rows_list)

df_seq_errors['labels'] = pd.Series(y_validate)
df_seq_errors['preds'] = pd.Series(y_predicted)
zeros_arr = np.zeros( y_predicted.shape[0], dtype=int)
df_seq_errors['errors'] = pd.Series(zeros_arr)  
df_seq_errors.loc[df_seq_errors['labels'] != df_seq_errors['preds'], 'errors'] = 1 
df_seq_errors['noises'] = pd.Series(noise_arr)  

# print(df_seq_errors.info())


# Rezultatų pasiskirstymas per pacientus

print("\nRezultatų pasiskirstymas per pacientus")

# Pasiruošimas
class_names = ['N', 'S', 'V']
n_classes = len(class_names)
df_user_errors = pd.DataFrame({'userNr':pd.Series(dtype='int'), 'userId':pd.Series(dtype='str'), 
    'N':pd.Series(dtype='int'), 'S':pd.Series(dtype='int'), 'V':pd.Series(dtype='int'),
    'Nprec':pd.Series(dtype='float') , 'Nrec':pd.Series(dtype='float'), 'Nfsc':pd.Series(dtype='float'),
    'Sprec':pd.Series(dtype='float') , 'Srec':pd.Series(dtype='float'), 'Sfsc':pd.Series(dtype='float'),
    'Vprec':pd.Series(dtype='float') , 'Vrec':pd.Series(dtype='float'), 'Vfsc':pd.Series(dtype='float'), 
    'Err%':pd.Series(dtype='float'), 'Noise%':pd.Series(dtype='float')})
# Pandas Empty DataFrame with Specific Column Types
# https://sparkbyexamples.com/pandas/pandas-empty-dataframe-with-specific-column-types/

rows_list = []

# Išgauname pacientų vidinę (eilės nr) numeraciją
grouped  = df_seq_errors.groupby(['userNr'])
userNrs = list(grouped.groups.keys())
print(f'Pacientų: {len(userNrs)}\n')

for userNr in grouped.groups:
# https://stackoverflow.com/questions/62041850/looping-over-pandas-groupby-output-when-grouping-by-multiple-columns-and-missin

    y_test = df_seq_errors.loc[grouped.groups[userNr]]['labels'].to_numpy(dtype=int)
    y_pred = df_seq_errors.loc[grouped.groups[userNr]]['preds'].to_numpy(dtype=int)

    Err = get_error(y_test, y_pred)
    
    noise_arr = df_seq_errors.loc[grouped.groups[userNr]]['noises'].to_numpy(dtype=int)
    Noise = np.sum(noise_arr, axis=0)/noise_arr.shape[0]*100.

    # Testavimui
    # acc = accuracy_score(y_test, y_pred)
    # print(f"userNr: {userNr} recordingNr:  {recordingNr} Accuracy: {acc:.2f}")
    # cnf_matrix = confusion_matrix_modified(y_test, y_pred, n_classes)
    # show_confusion_matrix(cnf_matrix, class_names)
    # https://towardsdatascience.com/confusion-matrix-for-your-multi-class-machine-learning-model-ff9aa3bf7826

    prec,rec,fsc,sup = precision_recall_fscore_support(y_test, y_pred, labels=[0, 1, 2], zero_division=0)
    userId = get_userId(rec_dir, userNr)

    dict_user_errors = {'userNr':int(userNr),'userId':str(userId), 
    'N':sup[0], 'S':sup[1], 'V':sup[2],
    'Nprec':prec[0], 'Nrec':rec[0], 'Nfsc':fsc[0],
    'Sprec':prec[1], 'Srec':rec[1], 'Sfsc':fsc[1],
    'Vprec':prec[2], 'Vrec':rec[2], 'Vfsc':fsc[2], 
     'Err%':Err,  'Noise%':Noise
    }
    rows_list.append(dict_user_errors)

df_user_errors =  pd.DataFrame(rows_list) 

# Išvedame suformuotą masyvą
tit1 = f"{'userNr':>6} {'userId':>24} {'N':>8} {'S':>4} {'V':>4}"
tit2 = f"{'Nprec':>8} {'Nrec':>5} {'Nfsc':>5}"
tit3 = f"{'Sprec':>8} {'Srec':>5} {'Sfsc':>5}"
tit4 = f"{'Vprec':>8} {'Vrec':>5} {'Vfsc':>5} {'Err%':>8} {'Noise%':>8}"
print(tit1+tit2+tit3+tit4)

for idx, row in  df_user_errors.iterrows():
    str1 =f"{int(row['userNr']):>6} {str(row['userId']):>6} {int(row['N']):>8} {int(row['S']):4} {int(row['V']):4}" 
    str2 = f"{row['Nprec']:>8.2f} {row['Nrec']:5.2f} {row['Nfsc']:5.2f}"
    str3 = f"{row['Sprec']:>8.2f} {row['Srec']:5.2f} {row['Sfsc']:5.2f}"
    str4 = f"{row['Vprec']:>8.2f} {row['Vrec']:5.2f} {row['Vfsc']:5.2f} {row['Err%']:8.1f} {row['Noise%']:8.1f}"
    print(str1+str2+str3+str4)

filepath = Path(model_dir, 'Rezultatu_pasiskirstymas_per_pacientus.csv') 
df_user_errors.to_csv(filepath)    
print(f'\nRezultatų pasiskirstymas per pacientus įrašytas į:  {filepath}')

# print(df_user_errors)




Rezultatų pasiskirstymas per pacientus
Pacientų: 1

userNr                   userId        N    S    V   Nprec  Nrec  Nfsc   Sprec  Srec  Sfsc   Vprec  Vrec  Vfsc     Err%   Noise%
  1000 613b1d673d08d4d1f3cdc8f8      653   12    0    1.00  0.95  0.98    0.25  0.83  0.38    0.00  0.00  0.00      4.8     32.6

Rezultatų pasiskirstymas per pacientus įrašytas į:  model_cnn_fda_vu_v1/Rezultatu_pasiskirstymas_per_pacientus.csv


In [89]:
# Rezultatų pasiskirstymas per pacientus ir jų įrašus
# Skaičiuojama visoms 3 klasėms Precision(tikslumas), Recall (atgaminimas), Fscore (F rodiklis)
# 
# https://towardsdatascience.com/you-dont-always-have-to-loop-through-rows-in-pandas-22a970b347ac
# 
pd.set_option("display.max_rows", 6000, "display.max_columns", 18)
pd.set_option('display.width', 1000)

# Rezultatų pasiskirstymas per pacientų įrašus
print("\n\nRezultatų pasiskirstymas per pacientų įrašus")

# Pasiruošimas
class_names = ['N', 'S', 'V']
n_classes = len(class_names)
df_rec_errors = pd.DataFrame({'userNr':pd.Series(dtype='int'),
    'userId':pd.Series(dtype='str'), 'file_name':pd.Series(dtype='str'),
    'N':pd.Series(dtype='int'), 'S':pd.Series(dtype='int'), 'V':pd.Series(dtype='int'),
    'Nprec':pd.Series(dtype='float') , 'Nrec':pd.Series(dtype='float'), 'Nfsc':pd.Series(dtype='float'),
    'Sprec':pd.Series(dtype='float') , 'Srec':pd.Series(dtype='float'), 'Sfsc':pd.Series(dtype='float'),
    'Vprec':pd.Series(dtype='float') , 'Vrec':pd.Series(dtype='float'), 'Vfsc':pd.Series(dtype='float'), 
     'Err%':pd.Series(dtype='float'), 'Noise%':pd.Series(dtype='float')})
# Pandas Empty DataFrame with Specific Column Types
# https://sparkbyexamples.com/pandas/pandas-empty-dataframe-with-specific-column-types/

rows_list = []

# Sugrupuojame eilutes pagal 'userId','recordingId', suskaičiuojame, kiek kiekviename įraše
# iš viso yra klasifikuojamų sekų (labels) ir kiek padaryta klaidų (errors).
# Grupavimo objektą paverčiame į normalų dataframe objektą

grouped  = df_seq_errors.groupby(['userNr','file_name'])
print(f'Pacientų įrašų: {grouped.ngroups}')
# print(f'{grouped.size()=}')

for key in grouped.groups:
# https://stackoverflow.com/questions/62041850/looping-over-pandas-groupby-output-when-grouping-by-multiple-columns-and-missin

    # print(f'\nGroup: {key}\n{df_seq_errors.loc[grouped.groups[key]]}')
    userNr = key[0]
    file_name = key[1]
    userId = get_userId(rec_dir, userNr)

    y_test = df_seq_errors.loc[grouped.groups[key]]['labels'].to_numpy(dtype=int)
    y_pred = df_seq_errors.loc[grouped.groups[key]]['preds'].to_numpy(dtype=int)

    Err = get_error(y_test, y_pred)

    noise_arr = df_seq_errors.loc[grouped.groups[key]]['noises'].to_numpy(dtype=int)
    Noise = np.sum(noise_arr, axis=0)/noise_arr.shape[0]*100.


    # *********************** Testavimui *******************************************************************
    # acc = accuracy_score(y_test, y_pred)
    # print(f"userNr: {userNr} recordingNr:  {recordingNr} Accuracy: {acc:.2f}")
    # cnf_matrix = confusion_matrix_modified(y_test, y_pred, n_classes)
    # show_confusion_matrix(cnf_matrix, class_names)
    # https://towardsdatascience.com/confusion-matrix-for-your-multi-class-machine-learning-model-ff9aa3bf7826
    # *********************************************************************************************************

    prec,rec,fsc,sup = precision_recall_fscore_support(y_test, y_pred, labels=[0, 1, 2], zero_division=0)
    
    dict_rec_errors = {'userNr':int(userNr), 'userId':userId, 'file_name':file_name,
    'N':sup[0], 'S':sup[1], 'V':sup[2],
    'Nprec':prec[0], 'Nrec':rec[0], 'Nfsc':fsc[0],
    'Sprec':prec[1], 'Srec':rec[1], 'Sfsc':fsc[1],
    'Vprec':prec[2], 'Vrec':rec[2], 'Vfsc':fsc[2], 
     'Err%':Err, 'Noise%':Noise
    }
    rows_list.append(dict_rec_errors)

df_rec_errors =  pd.DataFrame(rows_list) 

# Išvedame suformuotą masyvą
grouped  = df_rec_errors.groupby('userNr')
for userNr, group in grouped:
    # print(group.dtypes)
    print("\n")
    userId = get_userId(rec_dir, userNr)
    print(f"{'userNr:'} {userNr} {'userId:'} {userId}" )
    tit1 = f"{'userNr':>6} {'file_name':>25} {'N':>8} {'S':>4} {'V':>4}"
    tit2 = f"{'Nprec':>8} {'Nrec':>5} {'Nfsc':>5}"
    tit3 = f"{'Sprec':>8} {'Srec':>5} {'Sfsc':>5}"
    tit4 = f"{'Vprec':>8} {'Vrec':>5} {'Vfsc':>5} {'Err%':>8} {'Noise%':>8}"
    print(tit1+tit2+tit3+tit4)

    for idx, row in group.iterrows():
        str1 =f"{int(row['userNr']):>6} {str(row['file_name']):>26} {int(row['N']):>8} {int(row['S']):4} {int(row['V']):4}" 
        str2 = f"{row['Nprec']:>8.2f} {row['Nrec']:5.2f} {row['Nfsc']:5.2f}"
        str3 = f"{row['Sprec']:>8.2f} {row['Srec']:5.2f} {row['Sfsc']:5.2f}"
        str4 = f"{row['Vprec']:>8.2f} {row['Vrec']:5.2f} {row['Vfsc']:5.2f} {row['Err%']:8.1f} {row['Noise%']:8.1f}"
        print(str1+str2+str3+str4)

filepath = Path(model_dir, 'Rezultatu_pasiskirstymas_per_irasus.csv') 
df_rec_errors.to_csv(filepath)    
print(f'\nRezultatų pasiskirstymas per įrašus įrašytas į:  {filepath}')




Rezultatų pasiskirstymas per pacientų įrašus
Pacientų įrašų: 1


userNr: 1000 userId: 613b1d673d08d4d1f3cdc8f8
userNr                 file_name        N    S    V   Nprec  Nrec  Nfsc   Sprec  Srec  Sfsc   Vprec  Vrec  Vfsc     Err%   Noise%
  1000                1631134.867      653   12    0    1.00  0.95  0.98    0.25  0.83  0.38    0.00  0.00  0.00      4.8     32.6

Rezultatų pasiskirstymas per įrašus įrašytas į:  model_cnn_fda_vu_v1/Rezultatu_pasiskirstymas_per_irasus.csv


In [90]:
# Sukuriami užduoto ilgio sekų vaizdai ir įrašomi į disko atitinkamus klasėms subfolderius

from zive_util_vu import get_rec_Id, split_seq_file_name, get_symbol_list, get_SubjCode, create_SubjCode
from zive_util_vu import read_show_seq_ext_zive, read_seq, get_seq_start_end, read_rec_attrib


def read_show_seq_ext_zive_npy(rec_dir, all_beats_attr, idx, win_ls, win_rs, win_ls_ext, win_rs_ext):
# Išpjauna užduoto ilgio seką iš mit2zive įrašo ir sukuria jos vaizdą su anotacijomis

# rec_dir - paciento EKG įrašų aplankas
# recordingId - paciento EKG įrašo Id - int
# i_sample - R dantelio, kurio atžvilgiu formuojama seka, indeksas viso EKG įrašo reikšmių masyve - int
# win_ls - klasifikuojamo EKG segmento plotis iki R pūpsnio (iš kairės) 
# win_rs - klasifikuojamo EKG segmento plotis nuo R pūpsnio (iš dešinės)
# win_ls_ext - vaizduojamo EKG segmento plotis iki R pūpsnio (iš kairės) 
# win_rs_ext - vaizduojamo EKG segmento plotis už R pūpsnio (iš dešinės) 

    ax = plt.gca()

    sequence, sample, label = read_seq(rec_dir, all_beats_attr, idx, win_ls_ext, win_rs_ext)
    if (sample == None):
        print("klaida!")
        return None
    
    seq_start = sample - win_ls_ext
    seq_end = sample + win_ls_ext

    SubjCode = get_SubjCode(idx, all_beats_attr)

    # Nuskaitome paciento anotacijas ir jų indeksus
    atr_sample, atr_symbol = read_rec_attrib(rec_dir, SubjCode)

    # # suformuojame anotacijų žymes
    beat_symbols,beat_locs = get_symbol_list(atr_symbol,atr_sample, seq_start, seq_end)

    # deltax ir deltay simbolių pozicijų koregavimui
    min = np.amin(sequence)
    max = np.amax(sequence)
    deltay = (max - min)/20
    deltax = len(sequence)/100

    # suformuojame vaizdą
    x = np.arange(0, len(sequence), 1)
    ax.plot(x, sequence, color="#6c3376", linewidth=2)
    left_mark = win_ls_ext - win_ls
    right_mark = win_ls_ext + win_rs
    ax.axvline(x = left_mark, color = 'b', linestyle = 'dotted')
    ax.axvline(x = right_mark, color = 'b', linestyle = 'dotted')
    for i in range(len(beat_locs)):
        ax.annotate(beat_symbols[i],(beat_locs[i]-deltax,sequence[beat_locs[i]]+deltay))
    ax.set_ylim([min, max+2*deltay])
    
    return(ax)

def get_seq_attributes(all_beats_attr, idx):
    # 'userNr', 'recordingNr', 'sample', 'symbol', 'label', 'RRl', 'RRr'
    row = all_beats_attr.loc[idx]
    return list(row) 

# ////////////////////// Užduodami parametrai /////////////////////

# Bendras aplankas vaizdams
images_folder = 'CNN'

# užduodame, kiek reikšmių vaizduosime prieš R dantelį ir po
wl_side_ext = 360
wr_side_ext = 360

# Užduodame, kiek sekų vaizdų iš kiekvienos klasės įrašysime į diską
img_max = 10

# /////////////////////////////////////////////////////////////////

print("\nBendras aplankas vaizdams:", images_folder)
print("\nMax sekų vaizdų skaičius iš kiekvienos klasės:", img_max)

if (img_max == 0):
    sys.exit()

# sukuriame bendrą aplanką vaizdams
images_dir = Path(sets_path, 'saved_images', images_folder)
create_dir(images_dir)

# klasių simbolinių vardų sąrašas
class_names = selected_beats.keys()

# sukuriame aplankus sekų vaizdams klasėse
create_subdir(images_dir, class_names)

# Sukuriame vaizdų klasėse skaitiklį
n_classes = len(class_names)
skait = np.zeros((n_classes, n_classes), dtype=int)
# print(skait)

# sukuriame selected_beats reversiją
rev_dict = get_rev_dictionary(selected_beats)

y_test = df_seq_errors['labels'].to_numpy(dtype=int)
y_pred = df_seq_errors['preds'].to_numpy(dtype=int)


# *********************** derinimui *******************************
# acc = accuracy_score(y_test, y_pred)
# print(f"\nAccuracy: {acc:.2f}\n")
# cnf_matrix = confusion_matrix_modified(y_test, y_pred, n_classes)
# show_confusion_matrix(cnf_matrix, class_names)
# ******************************************************************

# Ciklas per sekas

icycle = 0 
leng = len(y_pred)

for idx in validate_ind_lst:
    if (icycle >= leng):
        continue

# *********************** derinimui ******************************   
    # row = get_seq_attributes(all_beats_attr, idx)
    # print(idx, row)
# ****************************************************************

    # anotuotos klasės (klasių nr ir simboliniai pažymėjimai)
    label = y_test[icycle]
    label_symb = rev_dict[label]

    # print(f" {idx} label {label} symb {label_symb}")

    # klasifikatoriaus priskirtos klasės (klasių nr ir simboliniai pažymėjimai)
    pred = y_pred[icycle]
    pred_symb = rev_dict[pred]
    icycle +=1

    # print(f" {idx} pred {pred} symb {pred_symb}")

    # patikriname, ar neviršytas skaitiklis, jei viršytas, peršokame
    if (skait[label,pred] >= img_max):
        continue
    else:
        skait[label,pred] += 1

    SubjCode = get_SubjCode(idx, all_beats_attr)
    seq_name = str(SubjCode) + '_' + str(idx)
    
     # 'Išpjauname' užduoto ilgio sekas ir sukuriame jų vaizdus
    fig = plt.figure(facecolor=(1, 1, 1), figsize=(18,3))
    # print("rec_dir =", rec_dir)
    ax = read_show_seq_ext_zive_npy(rec_dir, all_beats_attr, idx, wl_side, wr_side, wl_side_ext, wr_side_ext) 
    if (ax == None):
        print(f'Sekai {idx} negali suformuoti išplėstinio vaizdo')
        plt.close()
        continue

    # suformuosime koreguotą failo vardą
    file_name = seq_name + '_' + pred_symb + ".png" 

    # suformuosime kelią į atitinkamą sub-aplanką
    image_subdir = Path(images_dir, label_symb)
    if (os.path.exists(image_subdir) == False):
        print('Klaida! ', image_subdir,' neegzistuoja')
    file_path = Path(image_subdir, file_name)
    # print('file_name: ',file_name, 'file_path: ', file_path)   

    # Įrašome į atitinkamą anotacijai sub-aplanką 
    ax.set_title(file_name)
    plt.savefig(file_path, bbox_inches='tight', pad_inches = 0.2)
    plt.close()

    # if (icycle >= len(y_pred)):
        # break

# ciklo per validate_ind_lst pabaiga

print("\n")
df = cm2df(skait, class_names)
print(df)

print("\nPabaiga.........")


ImportError: cannot import name 'split_seq_file_name' from 'zive_util_vu' (/home/kesju/DI/VSC/ZIVEIO/VU_APRIL/zive_util_vu.py)

In [ ]:
# Klasifikavimo rodiklių skaičiavimas vienam nurodytam įrašui


userNr, file_name = 1000, '1631134.867'

# userId, recordingId =  get_rec_Id(rec_dir, userNr, file_name)
print("\nZive įrašas:")
print(f"userNr: {userNr}  file_name: {file_name}")
# print(f"\nuserId: {userId}  recordingId: {recordingId}")

grouped  = df_seq_errors.groupby(['userNr','file_name'])
y_test = df_seq_errors.loc[grouped.groups[(userNr,file_name)]]['labels'].to_numpy(dtype=int)
y_pred = df_seq_errors.loc[grouped.groups[(userNr,file_name)]]['preds'].to_numpy(dtype=int)

print("\nAnotacijų pasiskirstymas įraše:")
cols, dist, tot = get_freq_unique_values(y_test, ['N', 'S', 'V'])
print(f"Klasės {cols}: {dist} Suma: {tot}\n ")

noise_arr = df_seq_errors.loc[grouped.groups[(userNr,file_name)]]['noises'].to_numpy(dtype=int)
Noise = np.sum(noise_arr, axis=0)/noise_arr.shape[0]*100.

Err = get_error(y_test, y_pred)

acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc:.2f} Error(%): {Err}\n ")
cnf_matrix = confusion_matrix_modified(y_test, y_pred, n_classes)
show_confusion_matrix(cnf_matrix, class_names)
# https://towardsdatascience.com/confusion-matrix-for-your-multi-class-machine-learning-model-ff9aa3bf7826
# *********************************************************************************************************

prec,rec,fsc,sup = precision_recall_fscore_support(y_test, y_pred, labels=[0, 1, 2], zero_division=0)
row = {
    'N':sup[0], 'S':sup[1], 'V':sup[2],
    'Nprec':prec[0], 'Nrec':rec[0], 'Nfsc':fsc[0],
    'Sprec':prec[1], 'Srec':rec[1], 'Sfsc':fsc[1],
    'Vprec':prec[2], 'Vrec':rec[2], 'Vfsc':fsc[2], 
     'Err%':Err, 'Noise%':Noise
    }

print("\n")
tit1 = f"{'N':>8} {'S':>4} {'V':>4}"
tit2 = f"{'Nprec':>8} {'Nrec':>5} {'Nfsc':>5}"
tit3 = f"{'Sprec':>8} {'Srec':>5} {'Sfsc':>5}"
tit4 = f"{'Vprec':>8} {'Vrec':>5} {'Vfsc':>5} {'Err%':>8} {'Noise%':>8}"
print(tit1+tit2+tit3+tit4)

str1 =f"{int(row['N']):>8} {int(row['S']):4} {int(row['V']):4}" 
str2 = f"{row['Nprec']:>8.2f} {row['Nrec']:5.2f} {row['Nfsc']:5.2f}"
str3 = f"{row['Sprec']:>8.2f} {row['Srec']:5.2f} {row['Sfsc']:5.2f}"
str4 = f"{row['Vprec']:>8.2f} {row['Vrec']:5.2f} {row['Vfsc']:5.2f} {row['Err%']:8.1f} {row['Noise%']:8.1f}"
print(str1+str2+str3+str4)




Zive įrašas:
userNr: 1000  file_name: 1631134.867

Anotacijų pasiskirstymas įraše:
Klasės ['N', 'S', 'V']: [653  12] Suma: 665
 
Accuracy: 0.95 Error(%): 4.8
 
Confusion Matrix
     N   S  V
N  623  30  0
S    0  10  2
V    0   0  0


Zero values! Cannot calculate Normalized Confusion Matrix


       N    S    V   Nprec  Nrec  Nfsc   Sprec  Srec  Sfsc   Vprec  Vrec  Vfsc     Err%   Noise%
     653   12    0    1.00  0.95  0.98    0.25  0.83  0.38    0.00  0.00  0.00      4.8     32.6
